# Wrangling, Summarizing, and Plotting 

## Preliminaries

This second tutorial goes a little bit further into ways to use `pandas` for taking early looks at and plotting data. Think about the many questions you might ask as you explore data. 

Think about the many types of explorative plots or summary statistics you might need to ask over and over again on eahc new dataset. You might want to group datapoints, or make early statistics like a *t-test* or make a plot of the distributions in the data.

`Pandas` provides ways to easily plot, summarize and organize the data so as to get you closer to answering initial questions on data.

In this tutorial, we will learn to analyze data by group. Specifically we will cover:

- the `split-apply-combine` concept
- grouping using `groupby()`
- doing simple grouped calculations and statistics 
- doing multiple calculations with `agg()`
- `multiIndexing` 
- simple summaries with `pivot` tables

We will use the dataset from before and load both `pandas` and `numpy`.

In [ ]:
import numpy as np
import pandas as pd

Let's re-use our function to load and clean up data from last time (but with a new name).

In [ ]:
def bcd_load_clean():
    bcd = pd.read_csv('./data/breast_cancer_data.csv')
    bcd['patient_id'] = bcd['patient_id'].astype('string')
    bcd['doctor_name'] = bcd['doctor_name'].str.split().str[1]
    bcd['bare_nuclei'] = bcd['bare_nuclei'].replace('?', '')
    bcd['bare_nuclei'] = pd.to_numeric(bcd['bare_nuclei'])
    
    return bcd

---

Load our data:

In [ ]:
bcd = bcd_load_clean()

---

Now, for convenience, let's make a smaller data set to play with. We'll do this by dropping some of the columns.

We can do this in one of two ways. We can either `.drop` the columns we don't want, or `.copy` the columns we do. Here's the first method:

In [ ]:
bcd2 = bcd.drop(labels = ['patient_id', 'cell_size_uniformity', 'cell_shape_uniformity', 
                          'cell_shape_uniformity', 'marginal_adhesion', 'single_ep_cell_size',
                          'bare_nuclei', 'normal_nucleoli', 'mitoses'],
                axis = 1,             # we're selecting column - default is rows
                inplace = False)      # we could modify bcd itself with True

In [ ]:
bcd2

---

In the cell below, make the same new data frame using column indexing and the `.copy()` method.

In [ ]:
 # make new bcd2 using .copy()

In [ ]:
 # look at new bcd2

---

## What might we want from this data set?

The main thing that comes to mind is whether any of the measures are related to the kind of tumor. To do this, we can

- group the data by the "class" column
- perform some operation, like computing the mean, separately for the groups.

We might also want to see if the doctors are behaving consistently with respect to one another.

### The split-apply-combine workflow

Much of data wrangling can be thought of "split-apply-combine". This is where we

- *split* the data into groups
- do ("*apply*") some function or manipulation on a per-group basis
- *combine* the results back into a data frame, series, etc.

Happily, the "combine" step is often handled for you by the methods that do the "apply" step.

#### Splitting - the `groupby()` method

Grouping the data is easy using the `groupby()` method. We just provide the name of a grouping variable. Since the main question at hand is how the measurements might relate to the type of tumor, Let's group by tumor "class".

In [ ]:
grpd = bcd2.groupby('class')

If we try to look at it:

In [ ]:
grpd

we see that the output of '.groupby() isn't a regular data frame, but rather a `DataFrameGroupBy` object. To interegate it, well need to use its methods or look at its attributes.

---

In the cell below, use the \<TAB\> key trick to browse the methods and properties that `grpd` has.

---

One of the early methods on the list is `.boxplot` - see what that does in the cell below!

While not the prettiest plot in the world, it does give us a hint that both of these variables might be related to tumor type.

(Make sure you remember or remind yourself what a box shows you.)

---

Now let's see if we can `.describe` the grouped data using the cell below.

In [ ]:
  # some useful summary numbers

---

What is the approximate mean difference between the groups for each of the two measures?  (no coding - just eyeball it)

---

Do a very rough guesstimate of Student's *t* for the clump thickness (e.g., just use the larger `std` and smaller `countn`). You can use the cell below as a calculator if you like. 

What does that tell you?

---

Now let's make a histogram of grouped data using the appropriate method.

Again, not the prettiest plot in the world. By comparison with the boxplot above, we can see that the top row of these histograms correspond to the benign tumors. We can also see that, consistent with the box plots, there is quite a bit overlap in the data values across groups. Is this consistent with your calculation of *t*?

---

We can also get the data for a specific group out of the grouped object. This will return a regular data frame the same width as the original, but only containing the requested group's data.

In [ ]:
grpd.get_group('benign')

---

In the cell below, confirm that the returned object is indeed a pandas `DataFrame`.

---

#### Applying - doing things to the data within groups

Once we have grouped data, we can easily caluculate things per group. Using the \<TAB\> trick, we can see that these objects produced by `groupby()` have methods for all the common statistical summaries.

##### *Simple calculations*

We can compute the mean for each measure *by group*

In [ ]:
my_means = grpd.mean(numeric_only = True)
my_means

---

In the cells below, compute and show the

standard deviations:

In [ ]:
my_sds =  # compute
 # show

and the counts

In [ ]:
 # compute
 # show

---

We can easily do simple maths on data frames of a compatible size. Here's a comparison of how many z-scores above zero each of the means are:

In [ ]:
my_zeds = my_means/my_sds
my_zeds

We can look at the difference between the z-scores for each measure using the `diff()` method, which takes the first difference down the rows.

In [ ]:
my_zeds.diff()

This difference in z-scores – how far apart two means are in terms of the standard deviation of the data – is roughly what statisticians call "effect size".

---

Why do the NaNs appear in the first row?

In the cell below, extract just the row with the numbers from `my_zeds` into a new series. (hint: using `.loc` is probably easiest)

In [ ]:
 # extract
 # show

---

We could also compute the difference by using `.loc[]` row indexing and simple maths.

In [ ]:
my_zeds.loc['malignant'] - my_zeds.loc['benign']

##### *Mulitiple caclulations with* `agg()`

We can do multiple calculation at once by placing function names inside the `agg()` ) or `aggregate()`) methods (they are synonyms). Here's where importing numpy comes in handy.

In [ ]:
grpd.agg([np.mean, np.std])

That either produced an error, or worked but pandas still complained to us because `grpd` has the doctors' names in it, and we obviously can't compute the means and standard deviations of those!

---

In the cell below, repeat the above calculation without triggering the warning.

---

We can use the pandas versions of functions by placing them in quotes. This is handy because, for example, pandas has a `count` and numpy doesn't.

In [ ]:
grpd[['clump_thickness', 'bland_chromatin']].agg([np.mean, 'std', 'count'])

#### MultiIndexing - getting at our summary data

Let's store our summary table little summary table above in its own data frame. This is going to complete our *split-apply-combine* by creating and naming a `DataFrame` object.

In [ ]:
my_summary = grpd[['clump_thickness', 'bland_chromatin']].agg([np.mean, 'std', 'count'])
my_summary

Notice that this data frame has hierachical column labels. In other words, there is a "clump_thickness" meta-column that contains three colums of its own, and that these subcolumns have the same names as those in the other meta-column. Thus saying "look at the mean column" would be ambiguous because the meta-column wasn't specified.

In pandas, this is known at "multiIndexing". 

Getting a meta-column is easy - it's just like getting a regular column from a data frame.

In [ ]:
my_summary['clump_thickness']

To get a subcolumn, we can index the meta-column, and then index the subcolumn from that.

In [ ]:
my_summary['clump_thickness']['mean']

If this looks confusing, consider the same thing broken up into two steps:

In [ ]:
meta_c = my_summary['clump_thickness']
meta_c

In [ ]:
meta_c['mean']

---

In the cell below, extract the mean and std of "bland_chromatin" in one go.

---

If we want values from a row, we need to get a bit more fancy and use `.loc`.

We can get a whole row using `df.loc[row_index(s)]`

In [ ]:
my_summary.loc['benign']

(note: that this gave us a hierarchical index!)

We can get a row and and a particular column with 
`df.loc[row_index(s),  (metacolumn_index, subcolumn_index)]`

In [ ]:
my_summary.loc['benign', ('clump_thickness', 'mean')]

We can also get bigger slices of the data with the colon `:` operator:

In [ ]:
my_summary.loc['benign', ('clump_thickness', 'mean'):('bland_chromatin', 'std')]

---

In the cell below, extract the mean and std of the bland chromatin meta-column.

---

#### Simple caculations with pivot tables

Pivot tables are summary data with the levels of one variable running down the row names (the index), the levels of another running across the column names, and values populating the interior. This should be made concrete by making one with the `pivot_table()` method:

In [ ]:
bcd2.pivot_table(index = 'doctor_name', columns = 'class', values = 'bland_chromatin')

By default, `pivot_table()` computes the group (row x column) means, but we can compute any of the standard summary statistics we wish. We just specify it using the `aggfunc` argument:

In [ ]:
bcd2.pivot_table(index = 'doctor_name', 
                 columns = 'class', 
                 values = 'bland_chromatin', 
                 aggfunc = 'std')

These are called "pivot tables" because their implementation makes it easy to pivot our view of the data summary.

---

In the cell below, "pivot" our view of the means so we have "class" down the rows, doctor name across the columns, and the means of clump thickness inside the table.

---

Because clumb thickness and bland chromatin only have the values 1 to 10, we could use either one as a grouping variable in a pivot table:

In [ ]:
bcd2.pivot_table(index = 'clump_thickness', columns = 'class', values = 'bland_chromatin')

Notice the NaNs – no benign tumors have a thickness of 9 or 10.

## Grouped plotting

As we have already seen, the seaborn package can take care of grouping for us – we just need to assign a grouping variable to color ('hue'), style, etc. And we already know how to do all this!

In [ ]:
import seaborn as sns

---

Make a distribution plot (like a histogram) of bland chromatin values grouped by tumor type.

---

Make a joint plot grouped by tumor type:

---

Make a pairplot of our two measurement variables grouped by tumor type and with `kind - 'hist'` in order to make histograms in the off-diagonal plots.

---

The plot above is okay except for the white space in the diagonal plots that aren't actually missing data. To fix this, we need to set our binwidth to 1, so that it matches the data (which are integers from 1 to 10).

Use the cell below to remake the plot with a binwidth of 1 for the diagonal plot. Hint: use the `plot_kws` argument to adjust this.

---

## Your conclusions

In the cell below, briefly state your conclusions from our analysis above. Are either or both of the measurements related to tumor type?

---

## Summary

In this tutorial, we learned to analyze data by group:

- the split-apply-combine concept
- grouping using `groupby()`
- doing simple grouped calculations 
- doing multiple calculations with `agg()`
- multiIndexing 
- simple summaries with pivot tables